In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import riskfolio as rp
import warnings
warnings.filterwarnings("ignore")

In [2]:
dows_df = pd.read_html('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average')[1]
dows_tickers = dows_df.Symbol.tolist()
dows_tickers.sort()
assert len(dows_tickers) == 30
SP500_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
SP500_tickers = SP500_df.Symbol.tolist()
SP500_tickers.remove('BF.B')  # delisted
SP500_tickers.remove('BRK.B')  # delisted
SP500_tickers.sort()

In [3]:
def download_data_and_calc_returns(tickers, period='5y', interval='1d', prepost=False):
    data = yf.download(tickers, period=period, interval=interval, prepost=prepost, threads=True)
    data = data.loc[:,('Adj Close', slice(None))]  # get adjusted close only
    data.columns = tickers
    data = data[tickers].pct_change().dropna()
    return data

dows_returns = download_data_and_calc_returns(dows_tickers)

[*********************100%***********************]  30 of 30 completed


In [4]:
dows_returns.head()

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-21,0.036830,0.004029,0.009479,-0.009198,0.007894,0.021038,0.012768,0.009383,-0.012092,-0.016466,...,0.010600,0.022975,0.015227,0.007570,0.023148,0.006068,0.013333,0.010751,0.012900,0.004258
2019-03-22,-0.020708,-0.027047,-0.021105,-0.028253,-0.032000,-0.032644,-0.022247,-0.022009,-0.003957,-0.007758,...,-0.007957,-0.026368,-0.066129,-0.008098,0.003944,-0.019562,-0.017522,0.025219,-0.018711,-0.007874
2019-03-25,-0.012091,-0.000589,-0.003837,0.022890,0.012407,-0.003839,-0.000190,-0.001625,-0.004066,0.011317,...,0.000729,0.005211,0.001703,0.002951,0.000371,-0.000931,-0.000261,0.005355,-0.011537,-0.001119
2019-03-26,-0.010332,0.009004,0.004219,-0.000216,0.003501,-0.009199,0.009482,0.010090,0.021802,-0.006104,...,0.006922,0.002125,0.012754,0.010396,0.000222,-0.014056,0.014834,0.009155,0.003729,0.001528
2019-03-27,0.008994,-0.010357,-0.004749,0.010341,-0.004854,-0.026851,-0.001691,-0.010795,0.001271,0.025589,...,-0.007598,-0.009668,-0.003478,-0.001165,0.010150,-0.006902,-0.006954,0.004123,0.005006,-0.011290


In [12]:
port = rp.Portfolio(returns=dows_returns, lowerret=pow(1.5, 0.2) - 1)
port.assets_stats()
risk_measure = 'SLPM'
objective = 'MinRisk'
risk_free_rate = 0.04
allocations = port.optimization(rm=risk_measure, obj=objective, rf=risk_free_rate)

The problem doesn't have a solution with actual input parameters


In [6]:
ax = rp.plot_pie(w=allocations, title='Sortino Min Risk', others=0.05, nrow=len(port.assetslist), cmap = "tab20", height=6, width=10, ax=None)

ValueError: w must be a DataFrame

In [ ]:
frontier = port.efficient_frontier(rm=risk_measure, points=100, rf=risk_free_rate)
display(frontier.T.head())

In [ ]:
# Plotting the efficient frontier
label = 'Sortino Min Risk Portfolio' # Title of point
mu = port.mu # Expected returns
cov = port.cov # Covariance matrix
returns = port.returns # Returns of the assets

ax = rp.plot_frontier(w_frontier=frontier, mu=mu, cov=cov, returns=returns, rm=risk_measure, alpha=0.05, cmap='viridis', rf=risk_free_rate, w=allocations, label=label, marker='*', s=16, c='r', height=6, width=10, ax=None)

In [ ]:
ax = rp.plot_frontier_area(w_frontier=frontier, cmap="tab20", height=6, width=10, ax=None)

In [ ]:
rp.Reports.excel_report(returns, allocations, rf=risk_free_rate, alpha=0.05, t_factor=252, ini_days=1, days_per_year=252, name='dows_1y_report')
rp.Reports.jupyter_report(returns, allocations, rm=risk_measure, rf=risk_free_rate, alpha=0.05, others=0.05, nrow=len(port.assetslist), height=6, width=14, t_factor=252, ini_days=1, days_per_year=252, bins=100)

In [ ]:

sp500_returns = download_data_and_calc_returns(SP500_tickers)
sp500_returns.head()

In [ ]:
port = rp.Portfolio(returns=sp500_returns, lowerret=pow(1.5, 0.2)-1)
port.assets_stats()
risk_measure = 'SLPM'
objective = 'MinRisk'
allocations = port.optimization(rm=risk_measure, obj=objective, rf=risk_free_rate)
ax = rp.plot_pie(w=allocations, title='Sortino Min Risk', others=0.05, nrow=len(port.assetslist), cmap="tab20", height=6, width=10, ax=None)

In [ ]:
frontier = port.efficient_frontier(rm=risk_measure, points=100, rf=risk_free_rate)
display(frontier.T.head())

In [ ]:
# Plotting the efficient frontier
label = 'Sortino Min Risk Portfolio'  # Title of point
mu = port.mu  # Expected returns
cov = port.cov  # Covariance matrix
returns = port.returns  # Returns of the assets
ax = rp.plot_frontier(w_frontier=frontier, mu=mu, cov=cov, returns=returns, rm=risk_measure, rf=risk_free_rate, alpha=0.05, cmap='viridis', w=allocations, label=label, marker='*', s=16, c='r', height=6, width=10, ax=None)

In [ ]:
ax = rp.plot_frontier_area(w_frontier=frontier, cmap="tab20", height=6, width=10, ax=None)

In [ ]:
rp.Reports.excel_report(returns, allocations, rf=risk_free_rate, alpha=0.05, t_factor=252, ini_days=1, days_per_year=252, name='sp500_1y_report')
rp.Reports.jupyter_report(returns, allocations, rm=risk_measure, rf=risk_free_rate, alpha=0.05, others=0.05, nrow=len(port.assetslist), height=6, width=14, t_factor=252, ini_days=1, days_per_year=252, bins=100)